In [1]:
import mysql.connector as connector
import pandas as pd

In [2]:
try:
    connection = connector.connect(
    user='root',
    password='******'
)
    print('Connected...')
except connector.Error as err:
    print('Error code: ', err.errno)
    print('Error msg: ', err.msg)

Connected...


In [3]:
try:
    cursor = connection.cursor()
    print('Cursor created...')
except:
    print('Its not possible to create the cursor!')

Cursor created...


In [19]:
cursor.execute("USE meta_py_to_mysql")
print('Database in use: ', connection.database)

Database in use:  meta_py_to_mysql


In [8]:
# 1. Create a stored procedure (without parameters) that make a join between 'bookings' and 'orders' tables to output the bill amount and the customer name:

cursor.execute("DROP PROCEDURE IF EXISTS getCustomersAndBillAmount") # Make sure the procedure do not exists before

stored_procedure_query = """
CREATE PROCEDURE getCustomersAndBillAmount()
BEGIN
SELECT b.bookingID, 
CONCAT(b.guestFirstName, ' ', b.guestLastName) AS customerName,
o.billAmount
FROM bookings b INNER JOIN orders o ON b.bookingID = o.bookingID 
ORDER BY billAmount DESC;
END
"""

cursor.execute(stored_procedure_query)
print("Procedure 'getCustomersAndBillAmount' created!")


Procedure 'getCustomersAndBillAmount' created!


In [21]:
# 1.1. Call the created procedure:

cursor.callproc("getCustomersAndBillAmount")
results1 = next(cursor.stored_results()) # next() function returns the next item in a list (it's like "manual for loop")
dataset1 = results1.fetchall()
columns1 = results1.column_names

df1 = pd.DataFrame(data=dataset1, columns=columns1)
df1.style.hide(axis='index')

bookingID,customerName,billAmount
1,Anna Iversen,2
2,Joakim Iversen,1
3,Vanessa McCarthy,1
5,Hiroki Yamane,1


In [26]:
# 2. Retrieve the booking ID, guest’s full name, and the bill amount of the top spender at closing.

cursor.execute("DROP PROCEDURE IF EXISTS TopSpender;")

procedure_topSpender = """
CREATE PROCEDURE TopSpender()
BEGIN
SELECT b.bookingID, CONCAT(b.guestFirstName,' ',b.guestLastName), MAX(o.billAmount) AS top_spender
FROM bookings b
INNER JOIN orders o
ON b.bookingID = o.bookingID;
END
"""
cursor.execute(procedure_topSpender)
print('Procedure created...')

Procedure created...


In [28]:
# 2.1. Call Stored procedure:

cursor.callproc("TopSpender")
results2 = next(cursor.stored_results())
dataset2 = results2.fetchall()
columns2 = results1.column_names

df2 = pd.DataFrame(data=dataset2, columns= columns2)
df2.style.hide(axis='index')

bookingID,customerName,billAmount
1,Anna Iversen,2
